##import library


In [1]:
!pip install PyPDF2==3.0.1 transformers accelerate torch huggingface_hub
import re
import pandas as pd
import requests
import torch
from PyPDF2 import PdfReader
from google.colab import files
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import BytesIO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

##upload files

In [2]:
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]
print("Dokumen :", csv_path)
df = pd.read_csv(csv_path)

Saving splp_perkara_202504150803.csv to splp_perkara_202504150803.csv
Dokumen : splp_perkara_202504150803.csv


In [3]:
df = df.head(10)

#passing to llm


In [4]:
notebook_login()

In [5]:
model_id = "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

##Function process

In [6]:
komisi_list = """
Berikut adalah daftar 13 Komisi DPR RI:
1. Komisi I: Pertahanan, luar negeri, intelijen, komunikasi dan informatika.
2. Komisi II: Pemerintahan dalam negeri, aparatur negara, dan reformasi birokrasi.
3. Komisi III: Hukum, HAM, dan keamanan.
4. Komisi IV: Pertanian, lingkungan hidup, kehutanan dan kelautan.
5. Komisi V: Infrastruktur dan transportasi.
6. Komisi VI: Perdagangan, koperasi, UMKM, dan BUMN.
7. Komisi VII: Energi, riset dan teknologi, dan lingkungan hidup.
8. Komisi VIII: Sosial, agama, dan pemberdayaan perempuan.
9. Komisi IX: Kesehatan, ketenagakerjaan, dan transmigrasi.
10. Komisi X: Pendidikan, olahraga, pariwisata, dan kebudayaan.
11. Komisi XI: Keuangan dan perbankan.
12. Komisi XII: Ekonomi kreatif dan digital.
13. Komisi XIII: Komunikasi, penyiaran dan literasi digital.
"""

In [7]:
def extract_text_from_pdf_bytes(pdf_path):
  reader = PdfReader(BytesIO(pdf_path))
  full_text = ""
  for page in reader.pages:
    full_text += page.extract_text()+"\n"
  return full_text

def extract_petitum_section(text):
  text_cleaned = re.sub(r'\s+', ' ', text, flags=re.MULTILINE).lower()

  match = re.search(r'(petitum.*?)pertimbangan hukum', text_cleaned, re.DOTALL)
  if match:
    petitum_text = match.group(1).strip()
    return petitum_text
  return None

def classify_petitum(petitum_text):
    prompt = f"""
Tentukan Teks berikut di bahas oleh Komisi DPR RI nomor berapa.
jawab hanya dengan nama komisi seperti 'komisi i', 'komisi ii', ..., sampai 'komisi xiii',
tanpa tambahan atau penjelasan apapun.Berikut adalah daftar komisi dan bidangnya :
{komisi_list}
Isi Teks:
{petitum_text}

"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    input_length = inputs['input_ids'].shape[1]
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.2,
        )
    generated_tokens = outputs[0][input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    match = re.search(r'\bkomisi\s+(?:[ivx]+)', generated_text, re.IGNORECASE)
    return match.group(0).strip() if match else generated_text.strip()

In [8]:
komisi_list_result = []

for i, row in df.iterrows():
    try:
        print(f"Memproses dokumen ke-{i+1}")
        url = row["supporting_file"]
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception("Gagal download PDF.")
        pdf_text = extract_text_from_pdf_bytes(response.content)
        petitum_text = extract_petitum_section(pdf_text)
        if not petitum_text:
            komisi_list_result.append("Permohonan gugur")
            continue
        klasifikasi = classify_petitum(petitum_text)
        komisi_list_result.append(klasifikasi)
    except Exception as e:
        komisi_list_result.append(f"Error: {str(e)}")

# Tambahkan hasil klasifikasi ke DataFrame dan simpan
df["komisi"] = komisi_list_result
df.to_csv("hasil_klasifikasi_petitum.csv", index=False)
print("✅ Proses selesai. File disimpan: hasil_klasifikasi_petitum.csv")

Memproses dokumen ke-1


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-2


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-3


...

Memproses dokumen ke-6


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-7


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-8
Memproses dokumen ke-9
Memproses dokumen ke-10
✅ Proses selesai. File disimpan: hasil_klasifikasi_petitum.csv


In [9]:
hasil_df = pd.read_csv("/content/hasil_klasifikasi_petitum.csv")
hasil_df

,id,putusancode,no_perkara,pokok_perkara,pemohon,amar_putusan,download_count,supporting_file,date,date_dpr,keywords,status,link_hit,status_dpr,user_input,tanggal_input,user_update,tanggal_update,komisi
0,1,4581,46/PUU-XXII/2024,Pengujian Materiil Undang-Undang Nomor 10 Tahu...,"Sahbirin Noor, S.Sos, M.H. (Pemohon I); Ahmad ...",Dalam Provisi: Menolak permohonan provisi Pemo...,1045,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T13:14:00+07:00,2024-07-30 13:14:00,masa jabatan kepala daerah pilkada serentak 2024,Menolak Sebagian (Tidak Dapat Diterima),https://api-test.mkri.id/putusan/download/4581,1,ibnu.hisyam,2024-08-07 10:30:11,ibnu.hisyam,2024-08-07 10:30:11,menimbang bahwa para pemohon mengajukan permoh...
1,2,4582,45/PUU-XXII/2024,Pengujian Materiil Undang-Undang Nomor 7 Tahun...,Ir. H. Didi Apriadi,Menolak permohonan Pemohon untuk seluruhnya.,939,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T12:48:00+07:00,2024-07-30 12:48:00,"disproporsionalitas hasil pemilu, Pasal 414 ay...",Menolak Seluruhnya,https://api-test.mkri.id/putusan/download/4582,1,ibnu.hisyam,2024-08-07 10:30:46,ibnu.hisyam,2024-08-07 10:30:46,Komisi II
2,3,4583,39/PUU-XXII/2024,Pengujian Materiil Undang-Undang Nomor 19 Tahu...,Muhammad Kahfi Andhika Bayu Adji,Menyatakan permohonan Pemohon tidak dapat dite...,259,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T12:35:00+07:00,2024-07-30 12:35:00,"kerugian minimal dalam korupsi, 1 milyar, koru...",Tidak Dapat Diterima,https://api-test.mkri.id/putusan/download/4583,1,ibnu.hisyam,2024-08-07 10:31:23,ibnu.hisyam,2024-08-07 10:31:23,Komisi III
3,4,4584,35/PUU-XXII/2024,Pengujian Materiil Undang-Undang Nomor 13 Tahu...,"Leonardo Olefins Hamonangan., S.H",Menolak permohonan Pemohon untuk seluruhnya.,609,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T12:26:00+07:00,2024-07-30 12:26:00,"batas usia, diskriminasi, ketenagakerjaan, low...",Menolak Seluruhnya,https://api-test.mkri.id/putusan/download/4584,1,ibnu.hisyam,2024-08-07 11:32:47,ibnu.hisyam,2024-08-07 11:32:47,Komisi III
4,5,4585,34/PUU-XXII/2024,Pengujian Materiil Kitab Undang- Undang Hukum ...,Sunyoto (Pemohon I) dan Jaka Fiton (Pemohon II),Menyatakan permohonan para Pemohon tidak dapat...,383,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T12:03:00+07:00,2024-07-30 12:03:00,"batas usia pensiun notaris, pembatasan wilayah...",Tidak Dapat Diterima,https://api-test.mkri.id/putusan/download/4585,1,ibnu.hisyam,2024-08-07 12:08:50,ibnu.hisyam,2024-08-07 12:08:50,Komisi X
5,6,4586,162/PUU-XXI/2023,Pengujian Materiil Undang-Undang Nomor 20 Tahu...,Djunatan Prambudi,Menolak permohonan Pemohon untuk seluruhnya.,291,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T11:48:00+07:00,2024-07-30 11:48:00,"syarat merek, kriteria pembeda merek",Menolak Seluruhnya,https://api-test.mkri.id/putusan/download/4586,1,ibnu.hisyam,2024-08-07 12:08:50,ibnu.hisyam,2024-08-07 12:08:50,"udian, pemohon juga menggugat pasal 21 ayat (1..."
6,7,4587,144/PUU-XXI/2023,Pengujian Materil Undang-Undang Nomor 20 Tahun...,Ricky Thio,1. Mengabulkan permohonan Pemohon untuk sebagi...,333,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T11:24:00+07:00,2024-07-30 11:24:00,"penghapusan merek, yang diajukan oleh pihak ke...",Mengabulkan Sebagian (Ditolak),https://api-test.mkri.id/putusan/download/4587,1,ibnu.hisyam,2024-08-07 12:10:21,ibnu.hisyam,2024-08-07 12:10:21,aga yang tidak jelas. c. bahwa kerugian konsti...
7,8,4588,57/PUU-XXII/2024,Pengujian Materiil Undang Undang Nomor 7 Tahun...,Pasai,Menyatakan permohonan Pemohon gugur.,278,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T10:42:00+07:00,2024-07-30 10:42:00,"ambang batas, pemilihan pimpinan DPR",Gugur,https://api-test.mkri.id/putusan/download/4588,1,ibnu.hisyam,2024-08-07 12:10:21,ibnu.hisyam,2024-08-07 12:10:21,Permohonan gugur
8,9,4589,53/PUU-XXII/2024,Pengujian Undang-Undang Nomor 1 Tahun 1970 ten...,"Dr. Charles Mangaraja Tampubolon, S.H.,